In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

PATH = "data/"
sz=299
arch=resnext101
architecture='resnext101'
bs=16

In [ ]:
label_csv='labels11.csv'
n = len(list(open(label_csv)))-1
print(n)
val_idxs = get_cv_idxs(n)


# Section for Additional Data

In [ ]:
import scipy.io
import re
train_list = scipy.io.loadmat('data/train1/train_list.mat')

test_list = scipy.io.loadmat('data/train1/test_list.mat')

file_list = scipy.io.loadmat('data/train1/file_list.mat')

In [ ]:
filenames=[(file_list)['file_list'][i][0][0][:-4] for i in range(len((file_list)['file_list']))]
re4="-[A-z]*\/"	# Variable Name 1p
#print(re.search(re4, sample).group(0)[1:-1])
labels=[(re.search(re4, filenames[i]).group(0)[1:-1]).lower() for i in range(len((file_list)['file_list']))]
additional_data=list(zip(filenames,labels))

In [ ]:
#df = pd.DataFrame(additional_data)
df = pd.DataFrame({'id':additional_data[]})#df = df[df.values != 0]
df.head()

In [ ]:
import csv

with open("output.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(additional_data)

import pandas as pd
# merge two CSV
a = pd.read_csv("labels.csv")
b = pd.read_csv("output.csv")
b = b.dropna(axis=1)
merged = a.merge(b,on='id')
merged.to_csv("final_label.csv", index=False)

# Training and data prep

In [7]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(path=PATH,
                                        folder='train',
                                        csv_fname=label_csv,
                                        bs=bs,
                                        tfms=tfms,
                                        val_idxs=val_idxs,
                                        test_name='test',
                                        suffix = '.jpg')


In [8]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=[0.5])

kaggle_class=data.classes

classs111=data.classes

for i in range(len(classs111)):
    if classs111[i] not in kaggle_class:
        print(classs111[i])

In [ ]:
len(data.classes)

In [ ]:
lr=learn.lr_find()
learn.sched.plot()


In [ ]:
learn.fit(1e-2,5) # 5starts to  Epoch overfitting


In [ ]:
learn.fit(1e-2,3) # 5starts to  Epoch overfitting


In [ ]:
lr=learn.lr_find()
learn.sched.plot()


In [ ]:
learn.fit(1e-5,2,cycle_len=3,cycle_mult=2) # 5starts to  Epoch overfitting


In [ ]:
learn.save(architecture+'Precompute_overfit_tmp')

In [10]:
learn.precompute=False

In [ ]:
lr=learn.lr_find()
learn.sched.plot()



In [ ]:
learn.fit(5e-4, 3, cycle_len=2) # 3 overfitting

In [ ]:
learn.fit(5e-4, 3, cycle_len=2) # 3 overfitting

In [ ]:
# Save Model Before Unfreezing the Layers
learn.save(architecture+'_'+str(sz)+'_Top')

In [11]:
learn.load(architecture+'_'+str(sz)+'_Top')

In [ ]:
# Unfreeze all Layers and Tune the Underlying Layers
learn.unfreeze()

lr=learn.lr_find()
learn.sched.plot()

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

 29%|██▉       | 431/1484 [11:50<28:56,  1.65s/it, loss=0.665]

In [ ]:
lr=np.array([1e-6,1e-4,1e-2])

In [ ]:
learn.fit(lr, 3, cycle_len=2)

In [ ]:
import datetime
import time
def Timestamp():
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%H-%M-%S')#'%Y%m%d-%H%M%S'
    return st


In [ ]:
print('{}_{}_{}'.format(architecture,sz,Timestamp()))

In [ ]:
learn.save('{}_{}_{}'.format(architecture,sz,Timestamp()))

In [ ]:
lr=learn.lr_find()
learn.sched.plot()

In [ ]:
lr=np.array([3e-5,3e-4,3e-3])

In [ ]:
%time learn.fit(lr, 3, cycle_len=2)
learn.save('{}_{}_{}'.format(architecture,sz,Timestamp()))

In [ ]:
lr=learn.lr_find()
learn.sched.plot()


In [ ]:
lr=np.array([5e-8,5e-6,5e-4])

In [ ]:
learn.fit(lr, 3, cycle_len=2)


In [ ]:
learn.save('{}_{}_{}'.format(architecture,sz,Timestamp()))

In [ ]:
learn.fit(lr, 10, cycle_len=2)


In [ ]:
learn.save('{}_{}_{}_final'.format(architecture,sz,Timestamp()))

## When to Stop Trainning ?


If training and validation are both low, you are probably under fitting and you can probably increase the capacity of your network and train more or longer (increase the number of epochs).
If there is an inflection point when training goes above the validation, you might be able to use early stopping(stop training).

In [ ]:
log_preds,y = learn.TTA()

## Analyzing results

In [ ]:
imr = ImageModelResults(data.val_ds, np.mean(log_preds,axis=0))
imr.num_classes

In [ ]:
data.classes

In [ ]:
imr.plot_most_uncertain(4)

In [ ]:
imr.plot_most_correct(0)

In [ ]:
imr.plot_most_incorrect(5)

In [ ]:
imr.plot_by_correct(5,is_correct=True)

In [ ]:
def calc_accuracy(y_pred,y_real):
    #from collections import Counter
    #Counter((y==preds))
    total=y_real.shape
    fail,crct=np.bincount(y_pred==y_real)
    return crct/total
    

In [ ]:
preds=np.argmax(np.mean(np.exp(log_preds),axis=0),axis=1)
calc_accuracy(preds,y)

In [ ]:
cnt=0
for i in range(len(y)):
    if(np.argmax(log_preds[:,i,:]).mean()==y[i]):
        cnt+=1
print('Accuracy : ',cnt/len(y))

In [ ]:
preds = np.argmax(log_preds, axis=1)
probs = np.exp(log_preds[:,1])

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

## Analyzing results

In [ ]:
num_classes = len(data.classes)
print(num_classes)

preds = np.argmax(log_preds, axis=1)
probs = np.exp(log_preds)

In [ ]:
is_correct=0
mult=1
idxs = np.where((preds == data.val_y)==is_correct & (data.val_y == y))[0]

print(np.argsort(mult * probs[idxs,y]))
#idxs[np.argsort(mult * probs[idxs,y])[:4]]

In [ ]:
def plot_val_with_title(idxs, title, y):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])    
    if type(y) == int: title_probs = [probs[x,y] for x in idxs]
    else:    
        key = 0;
        for x in idxs:
            title_probs = [probs[x,y[key]] for x in idxs]
            key += 1
    
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def most_by_mask(mask, y, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs,y])[:4]]

# Here the mult=-1 when the is_correct flag is true -> that means that when we want to display the most correct classes we will make a descending sorting (argsort) because we want that the biggest probabilities to be displayed first. 
# When is_correct is false, we want to display the most incorrect classes, so we want an ascending sorting since our interest is in the smallest probabilities.

def most_by_correct(y, is_correct): 
    mult = -1 if is_correct==True else 1
    return most_by_mask((preds == data.val_y)==is_correct & (data.val_y == y), y, mult)

In [ ]:
most_uncertain = np.argsort(np.average(np.abs(probs-(1/num_classes)), axis = 1))[:4]
idxs_col = np.argsort(np.abs(probs[most_uncertain,:]-(1/num_classes)))[:4,-1]
plot_val_with_title(most_uncertain, "Most uncertain predictions", idxs_col)

# for most correct classes with label 0
label = 0
#plot_val_with_title(most_by_correct(label, True), "Most correct class 0", label) 

## for most incorrect classes with label 2
#label = 2
#plot_val_with_title(most_by_correct(label, False), "Most incorrect class 2", label)

In [ ]:
imr=ImageModelResults(data.val_ds,log_preds)


In [ ]:
imr.plot_most_incorrect(0)

In [ ]:
plot_confusion_matrix(cm, data.classes)

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

def plot_val_with_title(idxs, title):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask((preds == data.val_y)==is_correct & (data.val_y == y), mult)

In [ ]:
plot_val_with_title(most_by_correct(0, False), "Most incorrect cats")

In [ ]:
plot_val_with_title(most_by_correct(1, False), "Most incorrect dogs")

# Predictions Submit to Kaggle

In [ ]:
learn.load('resnet50_299_Top')

In [ ]:
multi_preds, y = learn.TTA(is_test=True)
preds = np.mean(multi_preds, 0)

In [ ]:
preds=np.exp(preds)

In [ ]:
preds.shape

In [ ]:
columns = pd.read_csv('sample_submission.csv', index_col='id').columns
test = pd.DataFrame(preds)
test.columns = columns
test.index = [i.split('.jpg')[0].split('/')[-1] for i in data.test_dl.dataset.fnames]
test.index.name ='id'

In [ ]:
test.to_csv('FineTune_{}_with_data.csv'.format(architecture))

sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(5)